# 📝 2022년 데이터마이닝 기출문제 - 완벽 정리본
## ⭐ 시험장에서 바로 이해할 수 있도록 주석 완비!

### 문제 구성
- **문제 1**: 회귀분석 (Linear, Lasso, Ridge, ElasticNet) - 42점
- **문제 2**: 나이브 베이즈 + SVM - 25점

# 문제 1. 회귀분석 (주택 가격 예측) - 42점

## 🎯 목표: 주택 가격(Price) 예측
- **데이터**: HousePrices.csv
- **목표변수**: Price (주택 가격)
- **모델**: Linear Regression, Lasso, Ridge, ElasticNet
- **핵심 평가지표**: MAE (Mean Absolute Error)

## 📊 변수 설명
| 변수 | 설명 | 타입 |
|------|------|------|
| HomeID | 주택 번호 | 수치 (의미 없음) |
| Price | 주택 가격 | 수치 (목표변수) |
| SqFt | 주택 면적 (제곱피트) | 수치 |
| Bedrooms | 침실 수 | 수치 |
| Bathrooms | 화장실 수 | 수치 |
| Offers | 구매 제안 수 | 수치 |
| Brick | 벽돌 집 여부 | 범주형 (No, Yes) |
| Neighborhood | 동네 | 범주형 (East, North, West) |

## 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

# 회귀 모델
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 분류 모델 (문제 2용)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import f1_score, accuracy_score, classification_report

import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (선택사항)
try:
    from dmba import regressionSummary, classificationSummary
    HAS_DMBA = True
    print("✅ dmba 라이브러리 사용 가능")
except:
    HAS_DMBA = False
    print("⚠️ dmba 없음 - sklearn metrics만 사용")

## 데이터 로드 및 탐색

In [ ]:
# ========================================
# 데이터 로드
# ========================================

df = pd.read_csv('HousePrices.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 데이터 정보")
print("="*60)
df.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플")
print("="*60)
print(df.head())

print("\n" + "="*60)
print("📈 기본 통계")
print("="*60)
print(df.describe())

In [ ]:
# ========================================
# 변수 타입 자동 분류
# ========================================

print("="*60)
print("🔍 변수 타입 분류")
print("="*60)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n범주형 변수: {categorical_cols}")
print(f"수치형 변수: {numerical_cols}")

print("\n💡 HomeID는 주택 번호로 의미 없으므로 제외!")

## 문제 1-(1) 수치형 변수만 사용한 회귀분석

### 📌 요구사항
- 종속변수: Price
- 독립변수: **수치형 변수만** (HomeID 제외)
- **표준화 하지 않음** ← ⭐
- 데이터 분리: **8:2** (학습:테스트)
- 평가 지표: **MAE**

In [ ]:
# ========================================
# 1-(1) 독립변수/종속변수 분리 (수치형만)
# ========================================

print("="*60)
print("🔧 1-(1) 수치형 변수만 사용")
print("="*60)

# 목표변수
outcome = 'Price'

# 수치형 변수만 선택 (Price와 HomeID 제외)
numerical_predictors = [col for col in numerical_cols 
                       if col not in [outcome, 'HomeID']]  # ← HomeID 제외!

X_num = df[numerical_predictors]
y = df[outcome]

print(f"\n독립변수 (수치형만): {numerical_predictors}")
print(f"종속변수: {outcome}")
print(f"\n데이터 크기: {X_num.shape}")
print("\n💡 표준화 하지 않음!")

In [ ]:
# ========================================
# 학습/테스트 데이터 분리 (8:2)
# ========================================

X_train_num, X_test_num, y_train, y_test = train_test_split(
    X_num, y,
    test_size=0.2,      # ← ⭐ 테스트 20%
    random_state=42
)

print("\n" + "="*60)
print("✂️ 데이터 분리 (8:2)")
print("="*60)
print(f"학습 데이터: {X_train_num.shape} (80%)")
print(f"테스트 데이터: {X_test_num.shape} (20%)")

In [ ]:
# ========================================
# 회귀분석 모델 학습
# ========================================

print("\n" + "="*60)
print("📊 Linear Regression (수치형만)")
print("="*60)

lr_num = LinearRegression()
lr_num.fit(X_train_num, y_train)

print("✅ 모델 학습 완료!")

In [ ]:
# ========================================
# 예측 및 MAE 측정
# ========================================

# 예측
y_pred_train = lr_num.predict(X_train_num)
y_pred_test = lr_num.predict(X_test_num)

# MAE 계산
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

# 추가 지표
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

print("\n" + "="*60)
print("📊 문제 1-(1) 결과: MAE")
print("="*60)
print(f"\n[학습 데이터]")
print(f"MAE: ${mae_train:,.2f}")

print(f"\n[테스트 데이터]  ← ⭐ 답안!")
print(f"MAE:  ${mae_test:,.2f}  ← ⭐ 문제에서 요구한 지표!")
print(f"\n[추가 지표]")
print(f"RMSE: ${rmse_test:,.2f}")
print(f"R²:   {r2_test:.4f}")
print("="*60)

## 문제 1-(2) 10겹 교차검증 (수치형 변수만)

### 📌 요구사항
- 1-(1)과 동일한 변수 사용
- **10겹 교차검증** ← ⭐
- 평가 지표: **MAE**

In [ ]:
# ========================================
# 1-(2) 10겹 교차검증
# ========================================

print("="*60)
print("🔄 1-(2) 10겹 교차검증 (수치형만)")
print("="*60)

# 모델 생성
lr_cv = LinearRegression()

# 10겹 교차검증
cv_scores = cross_val_score(
    lr_cv,
    X_num,  # ← 전체 데이터 사용
    y,
    cv=10,                          # ← ⭐ 10겹
    scoring='neg_mean_absolute_error',  # ← ⭐ MAE (음수!)
    n_jobs=-1
)

# MAE로 변환 (음수 떼기)
mae_cv_scores = -cv_scores
mae_cv_mean = mae_cv_scores.mean()
mae_cv_std = mae_cv_scores.std()

print("\n" + "="*60)
print("📊 문제 1-(2) 결과: 10겹 CV MAE")
print("="*60)
print(f"10-Fold CV MAE: ${mae_cv_mean:,.2f} (±${mae_cv_std:,.2f})")
print(f"\n각 Fold MAE: {mae_cv_scores}")
print(f"\n평균 MAE: ${mae_cv_mean:,.2f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(3) 모든 변수 사용 + 10겹 교차검증

### 📌 요구사항
- 종속변수: Price
- 독립변수: **모든 변수** (HomeID 제외)
- **수치형: 표준화 안 함** ← ⭐
- **범주형: 원-핫 인코딩** ← ⭐
- **10겹 교차검증**
- 평가 지표: **MAE**

In [ ]:
# ========================================
# 1-(3) 모든 변수로 데이터 준비
# ========================================

print("="*60)
print("🔧 1-(3) 모든 변수 사용 (원-핫 인코딩)")
print("="*60)

# HomeID 제외한 모든 독립변수
all_predictors = [col for col in df.columns 
                 if col not in [outcome, 'HomeID']]

X_all = df[all_predictors].copy()

print(f"\n독립변수 (모든 변수): {all_predictors}")

# 범주형 변수 확인
categorical_in_X = [col for col in categorical_cols if col in X_all.columns]
print(f"\n범주형 변수: {categorical_in_X}")

# 원-핫 인코딩
if len(categorical_in_X) > 0:
    print(f"\n원-핫 인코딩 중...")
    X_all = pd.get_dummies(X_all, columns=categorical_in_X, drop_first=True)
    print(f"인코딩 후 변수 개수: {X_all.shape[1]}개")

print(f"\n최종 독립변수 목록: {X_all.columns.tolist()}")
print(f"최종 데이터 크기: {X_all.shape}")
print("\n💡 수치형 변수는 표준화하지 않음!")

In [ ]:
# ========================================
# 1-(3) 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 1-(3) 10겹 교차검증 (모든 변수)")
print("="*60)

# 모델 생성
lr_all = LinearRegression()

# 10겹 교차검증
cv_scores_all = cross_val_score(
    lr_all,
    X_all,
    y,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

# MAE로 변환
mae_cv_all = -cv_scores_all.mean()
std_cv_all = cv_scores_all.std()

print("\n" + "="*60)
print("📊 문제 1-(3) 결과: Linear Regression (모든 변수)")
print("="*60)
print(f"10-Fold CV MAE: ${mae_cv_all:,.2f} (±${std_cv_all:,.2f})")
print(f"\n평균 MAE: ${mae_cv_all:,.2f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(4) Lasso 회귀분석

### 📌 요구사항
- 1-(3)의 독립변수/종속변수 사용
- **RandomizedSearchCV** 사용 ← ⭐ Grid가 아니라 Randomized!
- alpha 범위: `np.linspace(0.0001, 1000, num=500)`
- **200번 반복** ← ⭐
- 평가 지표: `neg_mean_absolute_error`
- 최적 alpha로 **10겹 CV MAE** 측정

In [ ]:
# ========================================
# 1-(4) Lasso 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 1-(4) Lasso alpha 튜닝 (RandomizedSearchCV)")
print("="*60)

# alpha 후보 정의
alpha_range = np.linspace(0.0001, 1000, num=500)  # ← ⭐ 문제에서 지정!

param_dist_lasso = {
    'alpha': alpha_range
}

print(f"\nalpha 범위: {alpha_range.min():.4f} ~ {alpha_range.max():.2f}")
print(f"후보 개수: {len(alpha_range)}개")
print(f"랜덤 시도 횟수: 200번  ← ⭐")

# RandomizedSearchCV
random_search_lasso = RandomizedSearchCV(
    Lasso(max_iter=10000, random_state=42),
    param_dist_lasso,
    n_iter=200,                     # ← ⭐ 200번 반복
    cv=5,                           # CV는 기본 5겹
    scoring='neg_mean_absolute_error',  # ← ⭐ MAE
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_lasso.fit(X_all, y)

# 최적 alpha
best_alpha_lasso = random_search_lasso.best_params_['alpha']
best_mae_lasso = -random_search_lasso.best_score_

print("\n" + "="*60)
print("📊 Lasso 최적 alpha")
print("="*60)
print(f"최적 alpha: {best_alpha_lasso:.6f}  ← ⭐ 답안!")
print(f"CV MAE: ${best_mae_lasso:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 alpha로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 Lasso 10겹 교차검증 (최적 alpha)")
print("="*60)

# 최적 모델
lasso_best = Lasso(alpha=best_alpha_lasso, max_iter=10000, random_state=42)

# 10겹 교차검증
cv_scores_lasso = cross_val_score(
    lasso_best,
    X_all,
    y,
    cv=10,                          # ← ⭐ 10겹
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

mae_cv_lasso = -cv_scores_lasso.mean()
std_cv_lasso = cv_scores_lasso.std()

print("\n" + "="*60)
print("📊 문제 1-(4) 결과: Lasso")
print("="*60)
print(f"최적 alpha: {best_alpha_lasso:.6f}")
print(f"10-Fold CV MAE: ${mae_cv_lasso:,.2f} (±${std_cv_lasso:,.2f})")
print(f"\n평균 MAE: ${mae_cv_lasso:,.2f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(5) Ridge 회귀분석

### 📌 요구사항
- 1-(4)와 동일한 방식
- Ridge 모델 사용
- 최적 alpha로 **10겹 CV MAE** 측정

In [ ]:
# ========================================
# 1-(5) Ridge 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 1-(5) Ridge alpha 튜닝 (RandomizedSearchCV)")
print("="*60)

# alpha 후보 (Lasso와 동일)
param_dist_ridge = {
    'alpha': alpha_range
}

# RandomizedSearchCV
random_search_ridge = RandomizedSearchCV(
    Ridge(max_iter=10000, random_state=42),
    param_dist_ridge,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_ridge.fit(X_all, y)

# 최적 alpha
best_alpha_ridge = random_search_ridge.best_params_['alpha']
best_mae_ridge = -random_search_ridge.best_score_

print("\n" + "="*60)
print("📊 Ridge 최적 alpha")
print("="*60)
print(f"최적 alpha: {best_alpha_ridge:.6f}  ← ⭐ 답안!")
print(f"CV MAE: ${best_mae_ridge:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 alpha로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 Ridge 10겹 교차검증 (최적 alpha)")
print("="*60)

# 최적 모델
ridge_best = Ridge(alpha=best_alpha_ridge, max_iter=10000, random_state=42)

# 10겹 교차검증
cv_scores_ridge = cross_val_score(
    ridge_best,
    X_all,
    y,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

mae_cv_ridge = -cv_scores_ridge.mean()
std_cv_ridge = cv_scores_ridge.std()

print("\n" + "="*60)
print("📊 문제 1-(5) 결과: Ridge")
print("="*60)
print(f"최적 alpha: {best_alpha_ridge:.6f}")
print(f"10-Fold CV MAE: ${mae_cv_ridge:,.2f} (±${std_cv_ridge:,.2f})")
print(f"\n평균 MAE: ${mae_cv_ridge:,.2f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(6) ElasticNet 회귀분석

### 📌 요구사항
- alpha: `np.linspace(0.0001, 1000, num=500)`
- l1_ratio: `np.linspace(0, 1, num=500)` ← ⭐ 추가!
- **max_iter=100000** ← ⭐ 문제에서 지정!
- RandomizedSearchCV 200번
- 최적 alpha, l1_ratio로 **10겹 CV MAE** 측정

In [ ]:
# ========================================
# 1-(6) ElasticNet 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 1-(6) ElasticNet alpha & l1_ratio 튜닝")
print("="*60)

# 파라미터 후보 정의
alpha_range_elastic = np.linspace(0.0001, 1000, num=500)  # ← alpha
l1_ratio_range = np.linspace(0, 1, num=500)               # ← ⭐ l1_ratio

param_dist_elastic = {
    'alpha': alpha_range_elastic,
    'l1_ratio': l1_ratio_range
}

print(f"\nalpha 범위: {alpha_range_elastic.min():.4f} ~ {alpha_range_elastic.max():.2f}")
print(f"l1_ratio 범위: {l1_ratio_range.min():.1f} ~ {l1_ratio_range.max():.1f}")
print(f"max_iter: 100000  ← ⭐ 문제에서 지정!")
print(f"랜덤 시도 횟수: 200번")

# RandomizedSearchCV
random_search_elastic = RandomizedSearchCV(
    ElasticNet(
        max_iter=100000,  # ← ⭐ 문제에서 지정!
        random_state=42
    ),
    param_dist_elastic,
    n_iter=200,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_elastic.fit(X_all, y)

# 최적 파라미터
best_alpha_elastic = random_search_elastic.best_params_['alpha']
best_l1_ratio = random_search_elastic.best_params_['l1_ratio']
best_mae_elastic = -random_search_elastic.best_score_

print("\n" + "="*60)
print("📊 ElasticNet 최적 파라미터")
print("="*60)
print(f"최적 alpha: {best_alpha_elastic:.6f}  ← ⭐ 답안!")
print(f"최적 l1_ratio: {best_l1_ratio:.6f}  ← ⭐ 답안!")
print(f"CV MAE: ${best_mae_elastic:,.2f}")
print("="*60)

In [ ]:
# ========================================
# 최적 파라미터로 10겹 교차검증
# ========================================

print("\n" + "="*60)
print("🔄 ElasticNet 10겹 교차검증 (최적 파라미터)")
print("="*60)

# 최적 모델
elastic_best = ElasticNet(
    alpha=best_alpha_elastic,
    l1_ratio=best_l1_ratio,
    max_iter=100000,
    random_state=42
)

# 10겹 교차검증
cv_scores_elastic = cross_val_score(
    elastic_best,
    X_all,
    y,
    cv=10,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

mae_cv_elastic = -cv_scores_elastic.mean()
std_cv_elastic = cv_scores_elastic.std()

print("\n" + "="*60)
print("📊 문제 1-(6) 결과: ElasticNet")
print("="*60)
print(f"최적 alpha: {best_alpha_elastic:.6f}")
print(f"최적 l1_ratio: {best_l1_ratio:.6f}")
print(f"10-Fold CV MAE: ${mae_cv_elastic:,.2f} (±${std_cv_elastic:,.2f})")
print(f"\n평균 MAE: ${mae_cv_elastic:,.2f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(7) 모델 비교

### 📌 요구사항
- (3), (4), (5), (6)의 결과 비교
- Linear, Lasso, Ridge, ElasticNet 성과 비교

In [ ]:
# ========================================
# 1-(7) 모델 비교
# ========================================

print("="*60)
print("🏆 문제 1-(7) 결과: 모델 비교")
print("="*60)

# 결과 정리
comparison_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Lasso', 'Ridge', 'ElasticNet'],
    '10-Fold CV MAE': [
        mae_cv_all,
        mae_cv_lasso,
        mae_cv_ridge,
        mae_cv_elastic
    ]
})

# 순위 추가
comparison_df['Rank'] = comparison_df['10-Fold CV MAE'].rank()

# 정렬
comparison_df = comparison_df.sort_values('10-Fold CV MAE')

print("\n", comparison_df.to_string(index=False))

# 최고 모델
best_model_name = comparison_df.iloc[0]['Model']
best_model_mae = comparison_df.iloc[0]['10-Fold CV MAE']

print(f"\n" + "="*60)
print(f"✅ 최고 성능 모델: {best_model_name}")
print(f"✅ MAE: ${best_model_mae:,.2f}")
print("="*60)

print("\n💡 해석:")
print("- MAE가 작을수록 좋은 모델")
print("- Lasso/Ridge/ElasticNet은 규제로 과적합 방지")
print("- 데이터에 따라 최적 모델이 달라질 수 있음")

In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
bars = plt.bar(comparison_df['Model'], comparison_df['10-Fold CV MAE'])

# 최고 모델 강조
bars[0].set_color('green')
bars[0].set_alpha(0.8)

plt.xlabel('Model', fontsize=12, fontweight='bold')
plt.ylabel('10-Fold CV MAE ($)', fontsize=12, fontweight='bold')
plt.title('Model Comparison - MAE', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

# 문제 2. 나이브 베이즈 + SVM (소득 분류) - 25점

## 🎯 목표: 소득 금액 구분 (>50K vs <=50K)
- **데이터**: income_evaluation_sub.csv
- **목표변수**: income_category (새로 생성)
- **모델**: Naive Bayes, LinearSVC, SVC
- **핵심 평가지표**: F1 Score

## 📊 변수 설명
| 변수 | 설명 | 타입 |
|------|------|------|
| age | 나이 | 수치 |
| educationnum | 교육받은 년도 | 수치 |
| occupation | 직종 | 범주형 |
| race | 인종 | 범주형 |
| sex | 성별 | 범주형 |
| capitalgain | 금융수익 | 수치 |
| capitalloss | 금융손실 | 수치 |
| hoursperweek | 주당 근무시간 | 수치 |
| income | 소득 (>50K, <=50K) | 범주형 (원본) |

## 데이터 로드

In [ ]:
# ========================================
# 데이터 로드
# ========================================

df_income = pd.read_csv('income_evaluation_sub.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 Income 데이터 정보")
print("="*60)
df_income.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df_income.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플")
print("="*60)
print(df_income.head())

print("\n" + "="*60)
print("📈 income 분포")
print("="*60)
print(df_income['income'].value_counts())

## 문제 2-(1) income_category 생성

### 📌 요구사항
- income이 `>50K`이면 **1**
- income이 `<=50K`이면 **0**
- 새 컬럼: `income_category`

In [ ]:
# ========================================
# 2-(1) income_category 생성
# ========================================

print("="*60)
print("🔧 2-(1) income_category 생성")
print("="*60)

# income_category 생성
df_income['income_category'] = (df_income['income'] == '>50K').astype(int)

print("\n변환 규칙:")
print(">50K → 1")
print("<=50K → 0")

print("\n" + "="*60)
print("📊 income_category 분포")
print("="*60)
print(df_income['income_category'].value_counts())
print("\n비율:")
print(df_income['income_category'].value_counts(normalize=True))

print("\n✅ income_category 생성 완료!")

## 문제 2-(2) 데이터 전처리

### 📌 요구사항
- 목표변수: `income_category`
- 독립변수: 의미 없는 변수 제외 (income 제외)
- **수치형 변수: 표준화** ← ⭐
- **범주형 변수: 원-핫 인코딩** ← ⭐

In [ ]:
# ========================================
# 2-(2) 변수 분리 및 전처리
# ========================================

print("="*60)
print("🔧 2-(2) 데이터 전처리")
print("="*60)

# 목표변수
outcome_income = 'income_category'

# 독립변수 (income과 income_category 제외)
predictors_income = [col for col in df_income.columns 
                    if col not in [outcome_income, 'income']]

X_income = df_income[predictors_income].copy()
y_income = df_income[outcome_income]

print(f"\n독립변수: {predictors_income}")
print(f"종속변수: {outcome_income}")

# 변수 타입 분류
categorical_income = X_income.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_income = X_income.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n범주형 변수: {categorical_income}")
print(f"수치형 변수: {numerical_income}")

In [ ]:
# ========================================
# 범주형 변수 원-핫 인코딩
# ========================================

if len(categorical_income) > 0:
    print(f"\n원-핫 인코딩 중: {categorical_income}")
    X_income = pd.get_dummies(X_income, columns=categorical_income, drop_first=True)
    print(f"인코딩 후 변수 개수: {X_income.shape[1]}개")

print(f"\n인코딩 후 독립변수 목록 (처음 10개): {X_income.columns.tolist()[:10]}")

In [ ]:
# ========================================
# 수치형 변수 표준화
# ========================================

# 수치형 변수 다시 찾기 (인코딩 후)
numerical_in_X = [col for col in X_income.columns if col in numerical_income]

if len(numerical_in_X) > 0:
    scaler_income = StandardScaler()
    X_income[numerical_in_X] = scaler_income.fit_transform(X_income[numerical_in_X])
    print(f"\n✅ 표준화 완료: {numerical_in_X}")

print(f"\n최종 데이터 크기: {X_income.shape}")
print(f"최종 독립변수 개수: {X_income.shape[1]}개")

## 문제 2-(3) 나이브 베이즈 모델

### 📌 요구사항
- 나이브 베이즈 모델 생성
- **5겹 교차검증** ← ⭐
- 평가 지표: **F1 Score** ← ⭐

In [ ]:
# ========================================
# 2-(3) 나이브 베이즈 모델
# ========================================

print("="*60)
print("🔄 2-(3) 나이브 베이즈 (5겹 CV)")
print("="*60)

# 모델 생성 (GaussianNB - 표준화된 데이터용)
nb_model = GaussianNB()

# 5겹 교차검증
cv_scores_nb = cross_val_score(
    nb_model,
    X_income,
    y_income,
    cv=5,           # ← ⭐ 5겹
    scoring='f1',   # ← ⭐ F1 Score
    n_jobs=-1
)

f1_cv_nb = cv_scores_nb.mean()
std_cv_nb = cv_scores_nb.std()

print("\n" + "="*60)
print("📊 문제 2-(3) 결과: Naive Bayes")
print("="*60)
print(f"5-Fold CV F1 Score: {f1_cv_nb:.4f} (±{std_cv_nb:.4f})")
print(f"\n각 Fold F1: {cv_scores_nb}")
print(f"\n평균 F1 Score: {f1_cv_nb:.4f}  ← ⭐ 답안!")
print("="*60)

## 문제 2-(4) LinearSVC (하이퍼파라미터 튜닝)

### 📌 요구사항
- LinearSVC 모델
- **C 하이퍼파라미터 튜닝** ← ⭐
- **5겹 교차검증**
- 평가 지표: **F1 Score**
- 💡 학습 시간이 길어서 튜닝 횟수 적게!

In [ ]:
# ========================================
# 2-(4) LinearSVC C 튜닝
# ========================================

print("="*60)
print("🔧 2-(4) LinearSVC C 튜닝 (RandomizedSearchCV)")
print("="*60)

# C 후보 정의 (적은 개수)
param_dist_svc = {
    'C': np.logspace(-3, 3, 20)  # 20개 후보
}

print(f"\nC 범위: {param_dist_svc['C'].min():.4f} ~ {param_dist_svc['C'].max():.2f}")
print(f"랜덤 시도 횟수: 10번 (학습 시간 고려)")

# RandomizedSearchCV (빠른 튜닝)
random_search_svc = RandomizedSearchCV(
    LinearSVC(
        max_iter=10000,
        random_state=42,
        dual=False  # ← 경고 방지
    ),
    param_dist_svc,
    n_iter=10,          # ← 10번만 (시간 고려)
    cv=5,
    scoring='f1',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n⏳ RandomizedSearchCV 진행 중...")
random_search_svc.fit(X_income, y_income)

# 최적 C
best_C_svc = random_search_svc.best_params_['C']
best_f1_svc = random_search_svc.best_score_

print("\n" + "="*60)
print("📊 문제 2-(4) 결과: LinearSVC")
print("="*60)
print(f"최적 C: {best_C_svc:.6f}  ← ⭐ 답안!")
print(f"5-Fold CV F1 Score: {best_f1_svc:.4f}  ← ⭐ 답안!")
print("="*60)

## 문제 2-(5) SVC (rbf 커널)

### 📌 요구사항
- SVC 모델
- **kernel='rbf'** ← ⭐ 문제에서 지정!
- **gamma=5** ← ⭐ 문제에서 지정!
- **C=0.001** ← ⭐ 문제에서 지정!
- **5겹 교차검증**
- 평가 지표: **F1 Score**

In [ ]:
# ========================================
# 2-(5) SVC (rbf 커널)
# ========================================

print("="*60)
print("🔄 2-(5) SVC (rbf 커널)")
print("="*60)

# 모델 생성 (파라미터 고정)
svc_model = SVC(
    kernel='rbf',   # ← ⭐ 문제에서 지정!
    gamma=5,        # ← ⭐ 문제에서 지정!
    C=0.001,        # ← ⭐ 문제에서 지정!
    random_state=42
)

print(f"\n설정 파라미터:")
print(f"- kernel: rbf")
print(f"- gamma: 5")
print(f"- C: 0.001")

# 5겹 교차검증
cv_scores_svc = cross_val_score(
    svc_model,
    X_income,
    y_income,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

f1_cv_svc = cv_scores_svc.mean()
std_cv_svc = cv_scores_svc.std()

print("\n" + "="*60)
print("📊 문제 2-(5) 결과: SVC (rbf)")
print("="*60)
print(f"5-Fold CV F1 Score: {f1_cv_svc:.4f} (±{std_cv_svc:.4f})")
print(f"\n각 Fold F1: {cv_scores_svc}")
print(f"\n평균 F1 Score: {f1_cv_svc:.4f}  ← ⭐ 답안!")
print("="*60)

## 문제 2 모델 비교 (추가)

In [ ]:
# ========================================
# 문제 2 모델 비교
# ========================================

print("="*60)
print("🏆 문제 2 모델 비교")
print("="*60)

# 결과 정리
comparison_df_2 = pd.DataFrame({
    'Model': ['Naive Bayes', 'LinearSVC', 'SVC (rbf)'],
    '5-Fold CV F1 Score': [
        f1_cv_nb,
        best_f1_svc,
        f1_cv_svc
    ]
})

# 순위
comparison_df_2['Rank'] = comparison_df_2['5-Fold CV F1 Score'].rank(ascending=False)
comparison_df_2 = comparison_df_2.sort_values('5-Fold CV F1 Score', ascending=False)

print("\n", comparison_df_2.to_string(index=False))

# 최고 모델
best_model_2 = comparison_df_2.iloc[0]['Model']
best_f1_2 = comparison_df_2.iloc[0]['5-Fold CV F1 Score']

print(f"\n" + "="*60)
print(f"✅ 최고 성능 모델: {best_model_2}")
print(f"✅ F1 Score: {best_f1_2:.4f}")
print("="*60)

In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
bars = plt.bar(comparison_df_2['Model'], comparison_df_2['5-Fold CV F1 Score'])

# 최고 모델 강조
bars[0].set_color('green')
bars[0].set_alpha(0.8)

plt.xlabel('Model', fontsize=12, fontweight='bold')
plt.ylabel('5-Fold CV F1 Score', fontsize=12, fontweight='bold')
plt.title('Model Comparison - F1 Score', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

# 🎯 최종 정리 및 답안 요약

## 문제 1. 회귀분석 (주택 가격 예측)

### (1) 수치형 변수만 - 테스트 MAE
- **답안**: 테스트 MAE 값 확인

### (2) 수치형 변수만 - 10겹 CV MAE
- **답안**: 10-Fold CV MAE 평균 확인

### (3) 모든 변수 - Linear Regression 10겹 CV MAE
- **답안**: 10-Fold CV MAE 평균 확인

### (4) Lasso
- **최적 alpha**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (5) Ridge
- **최적 alpha**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (6) ElasticNet
- **최적 alpha**: 출력된 값 확인
- **최적 l1_ratio**: 출력된 값 확인
- **10-Fold CV MAE**: 출력된 값 확인

### (7) 모델 비교
- **답안**: 비교 테이블 및 최고 모델 확인

---

## 문제 2. 나이브 베이즈 + SVM (소득 분류)

### (1) income_category 생성
- **답안**: >50K → 1, <=50K → 0

### (2) 데이터 전처리
- **수치형**: 표준화 완료
- **범주형**: 원-핫 인코딩 완료

### (3) Naive Bayes
- **답안**: 5-Fold CV F1 Score 평균 확인

### (4) LinearSVC
- **최적 C**: 출력된 값 확인
- **5-Fold CV F1 Score**: 출력된 값 확인

### (5) SVC (rbf)
- **답안**: 5-Fold CV F1 Score 평균 확인

---

## 💡 시험 팁

1. **⭐ 표시** 찾기 → 문제 핵심!
2. **평가 지표 주의** → MAE vs F1 Score
3. **교차검증 횟수** → 5겹 vs 10겹
4. **표준화 여부** → 문제마다 다름!
5. **RandomizedSearchCV** → n_iter 확인
6. **회귀는 음수 처리** 필수 (`-best_score_`)
7. **특정 파라미터 지정** → kernel, gamma, C 등